# A quick insight on Algebra and KZG Commitments

_(These contents do not attempt to be exhaustive. The main objective is to give the bare minimum background to understand KZG commitments)_
 
_**Disclaimer**: lack of formalism ahead!_

## Contents

1. [Preliminaries](#1.-Preliminaries)
    - 1.1 [Number Sets](#1.1-Number-Sets)
    - 1.2 [Prime Numbers](#1.2-Prime-Numbers)
    - 1.3 [Coprime Integers](#1.3-Coprime-Integers)
    - 1.4 [Modular aritmethic](#1.4-Modular-aritmethic)
    - 1.5 [Basic Algebraic Structures](#1.5-Basic-Algebraic-Structures)
        - 1.5.1 [Abelian Groups](#1.5.1-Abelian-Groups)
            - [Finite Groups, Cyclic Groups and Generators](#Finite-Groups,-Cyclic-Groups-and-Generators)
            - [The Exponential Map](#The-Exponential-Map)
            - [Pairings](#Pairings)
        - 1.5.2 [Rings](#1.5.2-Rings)
        - 1.5.3 [Fields & Galois Fields](#1.5.3-Fields-&-Galois-Fields)
    - 1.6 [Polynomials](#1.6-Polynomials)
2. [KZG Commitments](#2.-KZG-Commitments)
    - 2.1 [The Setup Phase](#2.1-The-Setup-Phase)
    - 2.2 [The Commitment Phase](#2.2-The-Commitment-Phase)
    - 2.3 [The Opening Phase](#2.3-The-Opening-Phase)
    - 2.4 [The Verification Phase](#2.4.-The-Verification-Phase)
    - 2.5 [A `sagemath` Implementation](#2.5-A-sagemath-Implementation)
3. [Closing remarks](#3.-Closing-remarks)
    - 3.1 [Recovering the message from the polynomial?](#3.1-Recovering-the-message-from-the-polynomial?)
    - 3.2 [Abstraction!](#3.2-Abstraction!)

# 1. Preliminaries

One can define Mathematics as the field that studies patterns, as it contains a vast amount of subfields, each of which ultimately leads to the description and formalization of patterns and behaviors under the formalism of axioms and theorems. In fact, the whole Mathematic universe sustains itself on a series of axioms (statements that are taken as true), being the most widely used the [Zermelo-Fraenkel axioms](https://en.wikipedia.org/wiki/Zermelo%E2%80%93Fraenkel_set_theory) (ZF axioms), which are the basis of modern Set Theory.

It is through set theory that we can define the most basic mathematical object: numbers.

## 1.1 Number Sets

We are going to start with the set of Natural numbers $\mathbb{N}$, which most authors define as the set of all positive integers: $\{1,2,3,4,\dots\}$ (we denote a set as a pair of $\{\}$ containing some/all of the elements of the set). If you're curious about how this set is constructed, check out [Wikipedia's article on the Set theoretic definition of natural numbers](https://en.wikipedia.org/wiki/Set-theoretic_definition_of_natural_numbers). Some mathematicians (like myself) would also include the number $0$ in this set, but for the sake of coherency with Least Authority's MoonMath manual, we'll stick to the former definition. In case one would want to explicitly include $0$ in the set of Natural numbers, some authors adopt the notation $\mathbb{N}_0$.

The set of Integers $\mathbb{Z}$ is an extension of $\mathbb{N}$ and is defined as the set of all positive and negative integers: $\{\dots,-3,-2,-1,0,1,2,3,\dots\}$. The need of this extension is due to the fact that some operations (like subtraction $^*$) are not closed under $\mathbb{N}$, meaning that the result of the operation is not necessarily an element of $\mathbb{N}$. More on that later.


The set of Rational numbers $\mathbb{Q}$ is an extension of $\mathbb{Z}$ and is defined as the set of all numbers that can be expressed as the quotient of two integers (assuming the divisor is non-zero): $\{\frac{a}{b} \mid a,b\in\mathbb{Z}, b\neq 0\}$ (this reads like _"the set composed by the quotients of $a$ and $b$, so that $a$ and $b$ are elements of the set of integers and $b$ is non-zero"_). This extension also arises from the need of closure under some operations (like division $^*$).

We are going to define the set of Irrational numbers (some authors denote it as $\mathbb{I}$) and the set of Real numbers $\mathbb{R}$ together. $\mathbb{I}$ is defined as the set of numbers that _"cannot be expressed as the quotient of two integers"_, and $\mathbb{R}$ can be easily defined as the union of $\mathbb{Q}$ and $\mathbb{I}$: $\mathbb{R} = \mathbb{Q}\cup\mathbb{I}$. Given that we won't need these number sets for this session, we won't touch on the construction of $\mathbb{R}$ and $\mathbb{I}$. If curious, you may check out [Wikipedia's article on the construction of the real numbers](https://en.wikipedia.org/wiki/Construction_of_the_real_numbers) and [Wikipedia's article on irrational numbers](https://en.wikipedia.org/wiki/Irrational_number).

Finally, another set that is of general interest in mathematics is the set of Complex numbers $\mathbb{C}$, which is defined as the set of numbers that can be expressed as $a+bi$, where $a,b\in\mathbb{R}$ and $i$ is the imaginary unit, defined as $i^2=-1$. This set extends the real numbers $\mathbb{R}$ and arises from the need of closure under some operations (like the square root of a negative number $^*$)

$^*$ _Soon you wil see that "subtraction", "division" and "square root" are actually aliases for something else..._ 😯

From now on, we'll be working with the set of Integers $\mathbb{Z}$ and the set of Natural numbers $\mathbb{N}$, so don't worry too much about the other sets.

## 1.2 Prime Numbers

A prime number is a natural Number ($\mathbb{N}$) that has the following properties:

- is greater than 1
- is divisible only by itself and $1$

_Remember: $a$ is divisible by $b$ if_ $\frac{a}{b}\in\mathbb{Z}$

We denote the set of prime numbers as $\mathbb{P}$. Note that there is only one even prime number: $2$. Several authors denote $\mathbb{P}_{\geq 3}$ as the set of **odd** prime numbers.

## 1.3 Coprime Integers

It is said that two integers $a$ and $b$ are coprime if their greatest common divisor (GCD) is $1$. We denote this as $gcd(a,b)=1$. This means that $a$ and $b$ do not share any common factors other than $1$.

For example $gcd(4,9)=1$ ($4$ and $9$) are coprime, but $gcd(4,8)=4$ ($4$ and $8$ are not coprime). Usually, this is checked by means of the [Euclidean algorithm](https://en.wikipedia.org/wiki/Euclidean_algorithm). A more brute-force way to check if two numbers are coprime is to check if their prime factorization has any common factors. Let's take the previous examples:

- $4$ and $9$ are coprime: $4=2^2$ and $9=3^2$, so they don't share any common factors
- $4$ and $8$ are not coprime: $4=2^2$ and $8=2^3$ because they share the common factor $2$ twice! $4=2^2$ and $8=2^3=2\cdot 2^2$, thus the GCD is $2^2=4$

A fun corollary of this is that if either $a$ or $b$ is prime, then $gcd(a,b)=1$, i.e. the pair is co-prime. This is because a prime number can only be divided by itself and $1$.

## 1.4 Modular aritmethic

Modular arithmetic is a system of arithmetic for integers, where numbers "wrap around" upon reaching a certain value—the modulus _(doesn't this sound familiar with computer science's concept of integer overflow/underflow? A Solidity `uint256` type is a positive integer defined on modular arithmetic of modulus $2^{256}$ !_ 🤯 _)_

For example, let's take modulus 5, and add the numbers 4 and 3:

$$4+3=7\equiv 2\quad\text{mod}~5$$

_(Note that the symbol $\equiv$ means "equivalent")_

So basically, $4$ plus $3$ is $7$, but if we "wrap around" the result to the modulus $5$, we get $2$. Another way to think about it is that every time we get a number that is greater than the modulus, we subtract the modulus until we get a number that is less than the modulus. i.e. we divide by the modulus and keep the remainder.

The exact same thing happens with multiplication:

$$2\cdot 4=8\equiv 3\quad\text{mod}~5$$

One can quickly notice that the modulus gives the number of possible results of the operation. In the previous example, we had $5$ possible results: $0,1,2,3,4$. So, if we are in modulus $n$ the set of numbers $\{0,\dots,n-1\}$ are actually all the numbers we will ever get when doing arithmetic operations with modulus $n$. This set is denoted as $\mathbb{Z_n}$ (other mathematicians, as myself, prefer denoting it as $\mathbb{Z}/n\mathbb{Z}$, but for the sake of coherency with Least Authority's MoonMath manual, we'll stick to the former notation).

These concepts give rise to **equivalence classes**. To put it easy, an equivalente class is a set of numbers that _"are equal"_ under modulus $n$. From each equivalente class, we take _"one representative number"_ to identify each class. Let's go back again to our modulus $5$ example. As mentioned, $\mathbb{Z}_5=\{0,1,2,3,4\}$. So, the equivalence class of $0$ is $\{\dots,-5,0,5,10,15,\dots\}$, basically $\{a\mid a\in\mathbb{Z}, \frac{a}{5}\in\mathbb{Z}\}$. What about the equivalence class of $3$? Well, it's $\{\dots,-2,3,8,13,18,\dots\}$, basically $\{a\mid a\in\mathbb{Z}, \frac{a - 3}{5}\in\mathbb{Z}\}$. It's the set of numbers that _"fall"_ on the same number once wrapping around 5 (i.e. the set of numbers that have the same remainder when divided by 5).

_(Note that the symbol $\in$ means "is an element of")_

If the equivalence classes remain unclear, think about a clock: it has $12$ hours, and once it reaches $12$, it goes back to $1$. So, the equivalence class of $1$ is $\{\dots,1,13,25,\dots\}$, the equivalence class of $2$ is $\{\dots,2,14,26,\dots\}$, and so on. The equivalence class of $12$<> is $\{\dots,0, 12,24,36,\dots\}$. That's why when a digital clock reads `13:00`, we say it's `1:00 PM`, because $13$ is equivalent to $1$ under modulus $12$.

These sets will give rise to interesting algebraic structures, as we'll see later.

## 1.5 Basic Algebraic Structures

So far, we've seen some basic number sets and some basic arithmetic operations. Now, we'll see some basic algebraic structures that arise from these sets and operations.

But first, what is Algebra? Many people think of Algebra as the study of equations, but that's not entirely true. Algebra is the study of mathematical structures, their properties and how to manipulate them. To put it simple, it is like having a _"bag of objects"_ and some _"rules"_ to manipulate those _"objects"_.

_(Doesn't this ring a bell for you? It's like having a `class` in Object Oriented Programming, and some `methods` to manipulate the `class` `instance` !_ 🤯 _)_

### 1.5.1 Abelian Groups

Any algebraic structure is defined through a set and an operation. Let's start with the most basic algebraic structure: the Abelian Group. An Abelian Group (also known as commutative group) $\mathbb{G}$ is a tuple $(G,\oplus)$ where $G$ is a set and $\oplus$ is a binary operation (i.e. an operation that takes two elements of the set and returns another element) that satisfies the following properties:

- Closure: $\forall a,b\in G, a\oplus b\in G$ (the elements returned by the operation are elements of the set).
- Associativity: $\forall a,b,c\in G, (a\oplus b)\oplus c=a\oplus(b\oplus c)$
- Identity element: $\exists e\in G, \forall a\in G, a\oplus e=e\oplus a=a$. $e$ is called the identity element. Sometimes it is also denoted by a **bold** "one": $\boldsymbol{1}$
- Inverse element: $\forall a\in G, \exists a^{-1}\in G, a\oplus a^{-1}=a^{-1}\oplus a=e$. $a^{-1}$ is called the inverse element of $a$.
- Commutativity: $\forall a,b\in G, a\oplus b=b\oplus a$

_(Note that the symbol $\forall$ means "for all" and $\exists$ means "there exists")_

If you're familiar with Object Oriented Programming, you can think of an Abelian Group as a `class` with a `method` that takes two `instances` of the `class` and returns another `instance` of the `class`. The `class` is the set $G$, the `method` is the binary operation $\oplus$, and the `instances` are the elements of the set $G$.

Let's see some examples of Abelian Groups:

- The set of integers $\mathbb{Z}$ with the binary operation $+$ (addition) is an Abelian Group. The identity element is $0$ and the inverse element of $a$ is $-a$. _(See why [earlier](#1.1-Number-Sets) we mentioned that subtraction is an alias? It is an alias for "addition with the inverse")_
- The set of integers $\mathbb{Z}$ with the binary operation $\cdot$ (product; multiplication) is **NOT** an Abelian Group. The identity element is $1$ but the inverse element of $a$ is $\frac{1}{a}$, which is NOT an element of $\mathbb{Z}$. _(Again, see why [earlier](#1.1-Number-Sets) we mentioned that division is an alias? It is an alias for "multiplication with the inverse")_.
- The set of rational numbers $\mathbb{Q}$ with the binary operation $\cdot$ (product; multiplication) is an Abelian Group. The identity element is $1$ and the inverse element of $a$ is $\frac{1}{a}$. _(See why [earlier](#1.1-Number-Sets) we mentioned that $\mathbb{Q}$ is an extension of $\mathbb{Z}$ ?)_

If the commutativity property is not present in the Abelian Group, then it is just called a Group. _Can you think of an example of a tuple $(G,\oplus)$ that is a Group but not an Abelian Group?_ 😉

#### Finite Groups, Group Order, Cyclic Groups and Generators

The previous examples of Abelian Groups are infinite (i.e. the set $G$ that composes $\mathbb{G}$ contains an infinite amount of elements). But what if the set $G$ is finite? Then we call it a Finite Group. The number of elements in the grpup $\mathbb{G}$ is called the order of the group, and is usually denoted by $\left|\mathbb{G}\right|$.

Let's go back to our example of the set of integers $\{0,1,2,3,4\}$ (which is a finite set) and the binary operation $+$. Is this a Finite Group? The answer is **NO**, because we lack **closure**: if we perform $2 + 3 = 5$, and $5$ is not in the set. However, if we change $+$ for $+$ under modulo $5$, then we do get a Finite Group, because this time $2 + 3 = 5\equiv 0\quad\text{mod}~5$. Indeed, all groups $(\mathbb{Z}_n, +~\text{mod}~ n)$ are finite (abelian) groups, whose order is $n$.

Let's talk about Cyclic groups. A Cyclic Group is a group that can be generated by a single element, called **generator** by means of the binary operation. For example, $\mathbb{Z}$ with the binary operation addition $+$ is indeed an infinite cyclic group, which has two different generators: $1$ and $-1$, because every integer can be generated by adding $1$ or $-1$ to itself, or through their inverses.

The same happens with $\mathbb{Z}_n$ with the binary operation addition $+$ under modulo $n$. For example, $\mathbb{Z}_5$ has four generators: $1$, $2$, $3$ and $4$, because every element of $\mathbb{Z}_5$ can be generated by adding $1$ or $4$ to itself. In fact, every group $(\mathbb{Z}_n, +~\text{mod}~ n)$ is a cyclic group, and the generators are co-prime with $n$.

For example, let's take the clock ($\mathbb{Z}_{12}$) with the binary operation addition $+$ under modulo $12$. The generators are $1$, $5$, $7$ and $11$. Try it out! By now you must have noticed that $\mathbb{Z}_p$ where $p\in\mathbb{P}$ (i.e. the set of prime numbers) with the binary operation addition $+$ under modulo $p$ is very interesting because it is a **finite cyclic abelian group** whose generators are all the numbers in the set except $0$ (because trivially all the numbers in the set are coprime with $p$ 😄).


#### The Exponential Map

Let's say we have a finite cyclic group $\mathbb{G}=(G,\oplus)$ of order $n$ ($\left|\mathbb{G}\right|=n$; it contains $n$ elements) whose generator is $g$. The exponential map is a function that takes an integer $a\in\mathbb{Z}_n$ and returns the element of the group that is generated by $g^a$. One can understand $g^a$ as applying $a$ times the $\oplus$ operation on $g$.

Let's dig into an example for clarity: take the finite cyclic abelian group $\mathbb{G}:=(\mathbb{Z}_5, +~\text{mod}~5)$ of order $5$, and let's choose the generator $2$ (you know it doesn't matter which one we choose because all the elements of $\mathbb{G}$ except $0$ are generators). The exponential map, which we'll call $\xi$ is:

\begin{align*}
\xi : \mathbb{Z}_5 &\rightarrow \mathbb{G}\\
a &\mapsto 2\cdot a\quad\text{mod}~5
\end{align*}

Let's test if this is true:

<center>

|     $\mathbb{Z}_5$         |         $\mathbb{G}$    |
|:---:|:----------------------------------------------:|
| $a$ |       $\xi(a)=2\cdot a\quad\text{mod}~5$       |
| $0$ |             $\xi(0)=2\cdot 0 = 0$              |
| $1$ |             $\xi(1)=2\cdot 1 = 2$              |
| $2$ |             $\xi(2)=2\cdot 2 = 4$              |
| $3$ | $\xi(3)=2\cdot 3 = 6\equiv 1\quad\text{mod}~5$ |
| $4$ | $\xi(4)=2\cdot 4 = 8\equiv 3\quad\text{mod}~5$ |

</center>

See? We have generated all the elements of $\mathbb{G}$ with the generator $2$ using the exponential map.

Let's dig into another example: take the finite cyclic abelian group $\mathbb{G}:=(\{1,2,3,4,5,6\}, \cdot~\text{mod}~7)$ of order $6$, and let's choose the generator $3$. The exponential map, which we'll call $\xi$ is:

\begin{align*}
\xi : \mathbb{Z}_6 &\rightarrow \mathbb{G}\\
a &\mapsto 3^a\quad\text{mod}~7
\end{align*}

Let's test if this is true:

<center>

|     $\mathbb{Z}_6$         |         $\mathbb{G}$     |
|:---:|:-----------------------------------------------:|
| $a$ |         $\xi(a) = 3^a\quad\text{mod}~7$         |
| $0$ |               $\xi(0)= 3^0 = 1$                 |
| $1$ |               $\xi(1)= 3^1 = 3$                 |
| $2$ |               $\xi(2)= 3^2 = 2$                 |
| $3$ |   $\xi(3)=3^3 = 27 \equiv 6\quad\text{mod}~7$   |
| $4$ |   $\xi(4)=3^4 = 81 \equiv 4\quad\text{mod}~7$   |
| $5$ | $\xi(5)=3^5 = 243 = 8\equiv 5\quad\text{mod}~7$ |

</center>

_(Note that the set $\{1,2,3,4,5,6\}$ can also be referred as $\mathbb{Z}_7^*$, because it's like $\mathbb{Z}_7$ but without the $0$)_

The exponential map allows to perform so-called _"calculations in the exponent"_. For the previous example, let's take that we want to compute $5\cdot 6\cdot 3\quad\text{mod}~7$. We can do it two ways:

- Directly: $5\cdot 6\cdot 3 = 90\equiv 6\quad\text{mod}~7$
- In the exponent: $5\cdot 6\cdot 3 \equiv 3^5\cdot 3^3\cdot 3^1 = 3^{5+3+1~\text{mod}~6}=3^3 = 27 \equiv 6$

This property gets more useful when we are dealing with large numbers, because we can significantly reduce the size of the numbers we are working with.

#### Pairings

A pairing map takes elements from a group $\mathbb{G}_1$ and elements from a group $\mathbb{G}_2$ and returns an element of a group $\mathbb{G}_T$. Let's define the pairing $e$, and a pair $(a,b)\mid a\in\mathbb{G_1}, b\in\mathbb{G}_2$:

\begin{align*}
e : \mathbb{G}_1\times\mathbb{G}_2 &\rightarrow \mathbb{G}_T\\
(a,b) &\mapsto e(a,b)
\end{align*}

A pairing is said to be **bilinear** if for all $g_1,g'_1\in\mathbb{G}_1$ and $g_2,g'_2\in\mathbb{G}_2$, it is satisfied that:

\begin{align*}
e(g_1\oplus g'_1,g_2) &= e(g_1,g_2)\oplus e(g'_1,g_2)\\
e(g_1,g_2\oplus g'_2) &= e(g_1,g_2)\oplus e(g_1,g'_2)
\end{align*}

A pairing is said to be **non-degenerate** if for all $g_1\in\mathbb{G}_1$ and $g_2\in\mathbb{G}_2$, it is satisfied that:

$$e(g_1,g_2)=\boldsymbol{1}_{\mathbb{G}_T}\Rightarrow \boldsymbol{1}_{\mathbb{G}_1}\quad\text{or}\quad\boldsymbol{1}_{\mathbb{G}_2}$$

In other words, if the pairing of two elements of $\mathbb{G}_1$ and $\mathbb{G}_2$ is the neutral element of $\mathbb{G}_T$, then at least one of the elements of $\mathbb{G}_1$ or $\mathbb{G}_2$ must be the neutral element of their respective group.

A simple example for a pairing is taking all the groups as the set of integers $\mathbb{Z}_p$ with the binary operation addition $+$ and the pairing $e$ as:

\begin{align*}
e : \mathbb{Z}\times\mathbb{Z} &\rightarrow \mathbb{Z}\\
(a,b) &\mapsto a\cdot b
\end{align*}

It is easy to see that

\begin{align*}
e(a + b, c) &= (a + b)\cdot c = a\cdot c + b\cdot c = e(a, c) + e(b, c)\\
e(a,b) &= 0 \Rightarrow a = 0 \quad\text{or}\quad b = 0
\end{align*}

both hold.

_Can you think of another example of a pairing?_ 😉

### 1.5.2 Rings

The next algebraic structure we'll see are Rings (more specifically, commutative rings). A Ring $R$ is a tuple $(R,\oplus,\otimes)$ where $R$ is a set and $\oplus$ and $\otimes$ are binary operations that satisfy the following properties:

- $(R,\oplus)$ is an Abelian Group
- Commutativity: $\forall a,b\in R, a\otimes b=b\otimes a$
- Associativity: $\forall a,b,c\in R, (a\otimes b)\otimes c=a\otimes(b\otimes c)$
- Distributivity: $\forall a,b,c\in R, (a\oplus b)\otimes c=(a\otimes c)\oplus(b\otimes c)$
- Identity element: $\exists e\in R, \forall a\in R, a\otimes e=e\otimes a=a$. $e$ is called the identity element. Sometimes it is also denoted by a **bold** "one": $\boldsymbol{1}$. In this case, the identity element for $\oplus$ can be denoted by a **bold** "zero": $\boldsymbol{0}$.

For example, the set of integers $\mathbb{Z}$ with the binary operations $+$ (addition) and $\cdot$ (product; multiplication) is a Ring. The identity element for $+$ is $0$ and the identity element for $\cdot$ is $1$.

### 1.5.3 Fields & Galois Fields

The final algebraic structure we'll see are Fields. A Field $\mathbb{F}$ is a tuple $(F,\oplus,\otimes)$ where $F$ is a set and $\oplus$ and $\otimes$ are binary operations that satisfy the following properties:

- $(F,\oplus,\otimes)$ is a Ring
- $\forall a\in F\setminus\{0\}, \exists a^{-1}\in F, a\otimes a^{-1}=a^{-1}\otimes a=e$. $a^{-1}$ is called the inverse element of $a$.

Galois Fields (also known as Finite Fields) are a special kind of Field, where the set $F$ is finite. The order of the field is the number of elements in the set $F$, and is usually denoted by $\left|\mathbb{F}\right|$. We are going to use what are known as Prime Fields, which are build by means of $\mathbb{Z}/p\mathbb{Z}$ with the binary operations $+$ (addition) and $\cdot$ (product; multiplication) under modulo $p$, where $p\in\mathbb{P}$. These special fields are denoted as $\mathbb{F}_p$.

_What if $\mathbb{Z}/n\mathbb{Z}$, where $n\notin\mathbb{P}$?_

In that case, we would have a Ring, but not a Field, because not every arbitrary element of $a\in\mathbb{Z}/n\mathbb{Z}\setminus\{0\}$ has an inverse $a^{-1}$ in the set. For example, let's take $\mathbb{Z}/6\mathbb{Z}$ with the binary operation $\cdot$ (product; multiplication) under modulo $6$. Since $\mathbb{Z}/6\mathbb{Z}=\{0,1,2,3,4,5\}$, let's construct the multiplication table:

<center>

| $\cdot$ | $0$ | $1$ | $2$ | $3$ | $4$ | $5$ |
|:-------:|:---:|:---:|:---:|:---:|:---:|:---:|
|   $0$   | $0$ | $0$ | $0$ | $0$ | $0$ | $0$ |
|   $1$   | $0$ | $1$ | $2$ | $3$ | $4$ | $5$ |
|   $2$   | $0$ | $2$ | $4$ | $0$ | $2$ | $4$ |
|   $3$   | $0$ | $3$ | $0$ | $3$ | $0$ | $3$ |
|   $4$   | $0$ | $4$ | $2$ | $0$ | $4$ | $2$ |
|   $5$   | $0$ | $5$ | $4$ | $3$ | $2$ | $1$ |

</center>

Notice that there are rows/columns (besides $0$) that do not have a single entry that equals 1? For example, $2$, $3$ and $4$. This means that only $1$ and $5$ have a multiplicative inverse in $\mathbb{Z}/6\mathbb{Z}$, while the rest of the elements do not. Therefore, $\mathbb{Z}/6\mathbb{Z}$ is not a Field. _What do the elements that do not have an inverse have in common?_ 😉

## 1.6 Polynomials

A polynomial is an expression consisting of variables (also called indeterminates) and coefficients, that involves only the operations of addition, subtraction, multiplication, and non-negative integer exponents of variables. An example of a polynomial of a single indeterminate $x$ is $x^2 - 4x + 7$. An example in three variables is $x^3 + 2xyz^2 - yz + 1$. Usually, when talking about polynomials, one refers to the set of polynomials with coefficients and values that "live" in the field of real numbers $\mathbb{R}$ (embeded with the binary operations $+$ (addition) and $\cdot$ (product; multiplication)). This set is denoted as $\mathbb{R}[x]$. For example, the polynomial $x^2 - 2$ is an element of $\mathbb{R}[x]$.

As you may have guessed, we can define polynomials over any field $\mathbb{F}$, and denote the set of polynomials with coefficients and values that "live" in $\mathbb{F}$ as $\mathbb{F}[x]$. For example, the polynomial $x^2 - 2$ is also an element of $\mathbb{F}_5[x]$.

The field over which we construct the polynomials greatly affects the properties of the polynomials. For example, let's take the polynomial $x^2 - 2$.
- If we are working with $\mathbb{R}[x]$, then this polynomial has two roots: $\sqrt{2}$ and $-\sqrt{2}$, because $x^2 - 2 = (x - \sqrt{2})(x + \sqrt{2})$.
- If we are working with $\mathbb{F}_5[x]$, then this polynomial has no roots, because $\pm\sqrt{2}$ are not elements of $\mathbb{F}_5$.

So, $x^2 - 2$ is an irreducible polynomial in $\mathbb{F}_5[x]$, but not in $\mathbb{R}[x]$.

Let's check out another example: the polynomial $x^2 + 1$.
- If we are working with $\mathbb{R}[x]$, then this polynomial has no roots, because $x^2 + 1 = (x + i)(x - i)$, where $i$ is the imaginary unit, and $i$ is not an element of $\mathbb{R}$.
- If we are working with $\mathbb{C}[x]$, then this polynomial has two roots: $\pm i$.
- If we are working with $\mathbb{F}_5[x]$, then this polynomial has two roots: $2$ and $3$, because $x^2 + 1 = (x - 2)(x - 3)$.

Let's elaborate a bit on the previous statement. Remember that in $\mathbb{F}_5[x]$, all operations are done under modulo $5$. So, let's expand $(x - 2)(x - 3)$:

\begin{align*}
(x - 2)(x - 3) &= x^2 - 3x - 2x + 6\\
&= x^2 - 5x + 6\\
&\equiv x^2 + 1\quad\text{mod}~5
\end{align*}

# 2. KZG Commitments

Now that we have a basic understanding of the algebraic structures we'll be working with, let's dive into the KZG Commitments. The KZG Commitment scheme is a commitment scheme that allows to commit to a polynomial $\phi(x) = a_0 + a_1x + a_2x^2+\dots+a_tx^t$, where $\phi(x)\in\mathbb{F}_p[x]$. _"To commit"_ means proving that you know the polynomial $\phi(x)$ without revealing it. Think of it as a sealed envelope.

## 2.1 The Setup Phase

Let's define a Galois Field $\mathbb{F}_p$ where $p$ is a prime number. We'll also define a maximum degree $t$ for the polynomials we'll be working with, so that $t < p$. _If you want to do this properly, you must choose a prime so that it fulfills the $k$ bits of security that you want to establish for your commitment scheme. For example, if you want $128$ bits of security, you must choose a prime $p$ so that_ $2^{128} < p < 2^{129}$.

Now, we'll define a generator $g$ of $\mathbb{F}_p$. We will also need to compute two groups $\mathbb{G}$ and $\mathbb{G}_T$ that have $g$ as their generator. We'll also need to define a pairing $e:\mathbb{G}\times\mathbb{G}\to\mathbb{G}_T$ that is bilinear and non-degenerate.

A trusted party then picks a random $\alpha\in\mathbb{F}_p^*$ and computes the public parameters $PP$:

$$
PP = \left\langle g, g^\alpha, g^{\alpha^2}, \dots, g^{\alpha^t}\right\rangle
$$

$PP$ will be publicly available **to everyone**. The trusted party will then **HAVE TO DELETE** $\alpha$. _(More on that later)_

## 2.2 The Commitment Phase

Now, let's say we want to commit to a polynomial $\phi(x) = a_0 + a_1x + a_2x^2+\dots+a_tx^t$, where $\phi(x)\in\mathbb{F}_p[x]$. We'll compute the commitment $\mathcal{C}\in\mathbb{F}_p$:

$$
\mathcal{C} = g^{\phi(\alpha)}
$$

You may ask youself, how? $\alpha$ is unknown! Well, remember that we have the public parameters $PP$, which contain $g^\alpha$, $g^{\alpha^2}$, $\dots$, $g^{\alpha^t}$. So, we can compute the commitment $\mathcal{C}$ by **evaluating on the exponent**:

\begin{align*}
\mathcal{C} &= g^{\phi(\alpha)}\\
&= g^{a_0 + a_1\alpha + a_2\alpha^2+\dots+a_t\alpha^t}\\
&= g^{a_0}\cdot g^{a_1\alpha}\cdot g^{a_2\alpha^2}\cdot\dots\cdot g^{a_t\alpha^t}\\
&= g^{a_0}\cdot (g^{\alpha})^{a_1}\cdot (g^{\alpha^2})^{a_2}\cdot\dots\cdot (g^{\alpha^t})^{a_t}\\
\end{align*}

Clearly, we know $g^\alpha, g^{\alpha^2},\dots$ from the Public Parameters ($PP$)! So, we can compute the commitment $\mathcal{C}$.

## 2.3 The Opening Phase

In this step, the Verifier will ask the Prover to "open" the commitment $\mathcal{C}$ to a random specific point $s\in\mathbb{F}_p$. In other words, the prover will have to evaluate $\phi(s)$ and commit the result in the form of the opening triplet $OT = \left(s,\phi(s),\mathcal{C}_{\psi_s}\right)$. Computing $\phi(s)$ is trivial, but how do we compute $\mathcal{C}_{\psi_s}$?

$\mathcal{C}_{\psi_s}$ is defined as $g^{\psi(\alpha)}$, where $\psi(\alpha)$ is the proof polynomial that is constructed from $\phi(x)$ and $s$. To contstruct this polynomial, let's imagine the polynomial $\rho(x)\in\mathbb{F}_p\mid \rho(x)=\phi(x)-\phi(s)$. Trivially, $\rho(x)$ has $s$ as one of its roots (because $\rho(s)=\phi(s)-\phi(s)=0$). That means it admits a factorization of the form $\rho(x)=\psi_s(x)\cdot(x-s)$, where $\psi_s(x)$ is a polynomial of degree $t - 1$.

Given that $\rho(s)=\phi(s)-\phi(s)=\psi_s(x)\cdot(x - s)$, we have $\psi_s(x)=\frac{\phi(x)-\phi(s)}{x - s}$. Now, we can compute the commitment $\mathcal{C}_{\psi_s}$ the same way as we computed the commitment $\mathcal{C}$, we just have to replace $\phi(x)$ with $\psi_s(x)$.

The opening triplet $OT$ is then sent to the Verifier.

## 2.4. The Verification Phase

The verifier has to check that the opening triplet $OT$ is valid. In other words, that $\phi(s)$ is indeed evaluating $\phi(x)$ (which is hidden in $\mathcal{C}$) on $s$.

The verifier knows $PP$ (public by default), $\mathcal{C}=g^{\phi(\alpha)}$, $s$, $\phi(s)$ and $\mathcal{C}_{\psi_s}=g^{\psi_s(\alpha)}$. Also, the verifier will make use of the pairing $e:\mathbb{G}\times\mathbb{G}\to\mathbb{G}_T$ that is bilinear and non-degenerate. Why? Imagine we had no pairing $e$, and the verifier attempted to verify the commitment $\mathcal{C}$:

\begin{align*}
\mathcal{C} &= g^{\phi(\alpha)}\\
&= g^{\psi_s(\alpha)\cdot(\alpha - s) + \phi(s)}\\
&= g^{\psi_s(\alpha)\cdot(\alpha - s)}\cdot g^{\phi(s)}\\
&= \mathcal{C}_{\psi_s}^{(\alpha - s)}\cdot g^{\phi(s)}
\end{align*}

This would precise the verifier knowing $\alpha$. But, since we have the pairing $e$, we can check if the following holds:

$$
e(\mathcal{C}, g) = e(\mathcal{C}_{\psi_s}, g^{\alpha - s})\cdot e(g^{\phi(s)}, g)
$$

Why? Let's elaborate:

\begin{align*}
e(\mathcal{C}, g) &= e(g^{\phi(\alpha)}, g)\\
&= e(g, g)^{\phi(\alpha)}\\
&= e(g, g)^{\psi_s(\alpha)\cdot(\alpha - s) + \phi(s)}\\
&= e(g, g)^{\psi_s(\alpha)\cdot(\alpha - s)}\cdot e(g, g)^{\phi(s)}\\
&= e(g^{\psi_s(\alpha)}, g^{(\alpha - s)})\cdot e(g, g)^{\phi(s)}\\
&= e(\mathcal{C}_{\psi_s}, g^{\alpha - s})\cdot e(g, g)^{\phi(s)}\\
&= e(\mathcal{C}_{\psi_s}, g^{\alpha}\cdot g^{- s})\cdot e(g, g)^{\phi(s)}
\end{align*}

The nice property is that the verifier has all the information needed to compute this. In particular, $g^{\alpha}$ is given in the Public Parameters ($PP$) and $g^{- s}$ is easily computable.


## 2.5 A `sagemath` Implementation

Let's replicate (and implement) the previous steps one by one.

> Remember, we want to prove that we know a Polynomial $\phi(x) = a_0 + a_1x + a_2x^2+\dots+a_tx^t$, where $\phi(x)\in\mathbb{F}_p[x]$.

### 2.5.1 Setup

First we'll have to define a couple variables:
- $p$ prime which will aid us to build our Prime Field $\mathbb{F}_p$
- $t$ the maximum degree of the polynomials in our Polynomial Field $\mathbb{F}_p[x]$, so that $t < p$
- We'll both take $\mathbb{G}$ and $\mathbb{G_T}$ as the Prime Field $\mathbb{F}_p$ (because it is cyclic and has a generator $g$).

Remember that our prime needs $k$ bits of security! So we'll have to define a function to get a safe random prime number

In [1]:
def safe_random_prime(k):
    """Generate a safe random prime.
    Code taken from
    https://ask.sagemath.org/question/44765/randomly-generate-a-safe-prime-of-given-length/?answer=44766#post-id-44766
    """
    while true:
        p = random_prime(2^k - 1, false, 2^(k - 1))
        if ZZ((p - 1)/2).is_prime():
            return p

k = 16 # 16 bits of security. My laptop is not beefy and I want my public parameters to be computed in a matter of seconds
p = safe_random_prime(k)
print(f"Our safe random prime is {p}")

Our safe random prime is 51347


Let's define the Prime Field $\mathbb{F}_p$

In [2]:
F_p = GF(p)
print(F_p)

Finite Field of size 51347


Let's generate a random $t$ so that $t < p$

In [3]:
t = 0

while t == 0:
    t = F_p.random_element()
print(f"The maximum degree is {t}")

The maximum degree is 37433


Now, let's get the generator $g$ of the Prime Field $\mathbb{F}_p$

In [4]:
g = F_p.multiplicative_generator()
print(f"The generator is {g}")

The generator is 2


Next, we have to define a pairing $e:\mathbb{G}\times\mathbb{G}\rightarrow\mathbb{G}_T$. We will define the simple pairing
$$
e(g_1,g_2)\mapsto g_1\cdot g_2 \quad\left(\text{mod}~p\right)
$$

In [5]:
def e(g1, g2):
    return F_p.prod((g1, g2))

We may also check that this pairing
- is bilinear:
$$
e(g_1^a,g_2^b)=e(g_1^{ab},g_2)=e(g_1,g_2^{ab})=e(g_1,g_2)^{ab}
$$
    see:
$$
ag_1\cdot bg_2 = abg_1\cdot g_2 = g_1\cdot abg_2 = ab(g_1\cdot g_2) \quad\left(\text{mod}~p\right)
$$ 
- non-degenerate
$$
e(g,g)\neq\boldsymbol{1}
$$
    see:
$$
2\cdot2 = 4 \neq 0 \quad\left(\text{mod}~p\right)
$$

At this point, the trusted party (us, haha) retrieves a random (and secret) $\alpha\in\mathbb{F}_p$

In [6]:
alpha = F_p.random_element()

This allows computing the Public Parameters `PP`

In [7]:
def compute_public_parameters(F_p, alpha, t):

    PP = []

    accumulated = 1
    for i in range(t + 1):
        PP.append(F_p.prod((
            accumulated, g
        )))
        accumulated = F_p.prod((accumulated, alpha))
    return PP

In [8]:
PP = compute_public_parameters(F_p, alpha, t)
print("The first elements of the public parameters are: {}".format(PP[:100]))

The first elements of the public parameters are: [2, 25632, 32953, 47920, 32600, 42408, 44280, 5436, 41244, 17086, 30568, 33225, 42276, 47019, 38459, 10691, 22060, 4378, 37524, 42929, 46306, 40417, 47083, 37131, 38247, 15090, 20638, 8211, 22173, 14870, 25203, 29018, 39714, 23160, 32900, 36183, 6571, 4856, 1932, 11258, 48805, 27073, 15889, 42569, 2429, 13782, 47779, 22689, 4163, 3475, 17751, 29606, 27513, 6759, 955, 18694, 48549, 32385, 8359, 19102, 40083, 28340, 28109, 45739, 13672, 24388, 7419, 38607, 7620, 47273, 7515, 36615, 48954, 36818, 31905, 18319, 17820, 41011, 9084, 16895, 47368, 44054, 35799, 14539, 44908, 43752, 16392, 19295, 48915, 50464, 31159, 8125, 49631, 35607, 18523, 13587, 13315, 18959, 4540, 8489]


Remember, once we have the public parameters, **we must delete `alpha`**.
_We are not going to do it though, for illustrative purposes!_

### 2.5.2 Commitment

It's time to do our commitment! In order to make it as applied as possible, let's commit a message. To do that though, we will some auxiliary code to help us!

#### Reed Solomon Codes

We're going to use Reed-Solomon encoding to transform our messages into a list of integers

In [9]:
from reedsolo import RSCodec

def reed_solomon_encode_string(input_string):

    # Initialize the Reed-Solomon encoder/decoder with a specific error correction level
    rs = RSCodec(10)  # You can adjust the error correction level as needed

    # Encode the input string
    encoded_bytes = rs.encode(input_string.encode())

    # Convert the encoded bytes to a list of integers
    return list(encoded_bytes)

We will also need to transform this into a list of tuples, which will contain coordinates

In [10]:
def message_as_coords(input_string):
    y = reed_solomon_encode_string(input_string)
    x = range(1, len(y) + 1)
    return list(zip(x, y))

Let's encode a message!

In [11]:
msg = message_as_coords("Hello Zepp!")
print(msg)

[(1, 72), (2, 101), (3, 108), (4, 108), (5, 111), (6, 32), (7, 90), (8, 101), (9, 112), (10, 112), (11, 33), (12, 25), (13, 255), (14, 128), (15, 142), (16, 165), (17, 163), (18, 57), (19, 193), (20, 169), (21, 195)]


#### Interpolating Polynomial
Cool! Let's create our Polynomial Ring $\mathbb{F}_p[x]$

In [12]:
F_p_X.<x> = PolynomialRing(F_p)
F_p_X

Univariate Polynomial Ring in x over Finite Field of size 51347

And get an interpolating polynomial over our coordinates (our message)

In [13]:
msg_poly = F_p_X.lagrange_polynomial(msg)
msg_poly

38784*x^20 + 29502*x^19 + 27814*x^18 + 15919*x^17 + 4780*x^16 + 35151*x^15 + 16478*x^14 + 40979*x^13 + 24598*x^12 + 21339*x^11 + 50472*x^10 + 6735*x^9 + 8685*x^8 + 9664*x^7 + 13815*x^6 + 14359*x^5 + 8374*x^4 + 37855*x^3 + 11600*x^2 + 18403*x + 26889

Since for $n$ points $(x,y)$ there exists one unique polynomial of degree $n-1$ that passes through all of them, we basically converted `"Hello Zepp"` into a polynomial!

---

Okay, now we have the polynomial that we want to commit. We will need to evaluate the polynomial at $\alpha$. We will "_evaluate it on the exponent_". We can write a function for that

In [14]:
from functools import reduce

def exponent_evaluation(PP, poly, g):
    # Recover the Prime Field
    F_p = poly.base_ring()
    # Get the Polynomial coefficients. They are given in ascending order
    poly_coefs = poly.coefficients()
    # Only keep the parameters needed
    pp_used = PP[:len(poly_coefs)]  
    
    return reduce(
        lambda a,b: a + (b[0]*b[1]),
        zip(poly_coefs, pp_used),
        0
    )

In [15]:
commitment = exponent_evaluation(PP, msg_poly, g)
commitment

29729

This is our commitment!

### 2.5.3 Opening

The Verifier asks the Prover to evaluate the Polynomial at a point $s\in\mathbb{F}_p$. So, let's choose this point randomly

In [16]:
s = F_p.random_element()
print(f"The point of evaluation `s` is {s}")

The point of evaluation `s` is 32493


Now we can evaluate our polynomial on `s`

In [17]:
poly_eval = msg_poly(s)
print(f"The evaluation is {poly_eval}")

The evaluation is 50946


Next we compute the proof polynomial

In [18]:
proof_poly = ((msg_poly - poly_eval)/(x - s)).numerator()
proof_poly

38784*x^19 + 28593*x^18 + 27545*x^17 + 6047*x^16 + 36329*x^15 + 5818*x^14 + 1098*x^13 + 32128*x^12 + 23845*x^11 + 42041*x^10 + 1750*x^9 + 28356*x^8 + 9625*x^7 + 212*x^6 + 21833*x^5 + 23876*x^4 + 9419*x^3 + 9955*x^2 + 44662*x + 508

And the corresponding opening commit

In [19]:
opening_commit = exponent_evaluation(PP, proof_poly, g)
opening_commit

11475

Now we have the opening triplet $\left(s,\phi(s),\mathcal{C}^{\psi_s(\alpha)}\right)$

In [20]:
opening_triplet = (s, poly_eval, opening_commit)
opening_triplet

(32493, 50946, 11475)

### 2.5.4 Verification

Now the Verifier has to check if $\phi(s)$ is indeed evaluating $\phi(x)$ on $s$

In [21]:
e(commitment, g) == (e(opening_commit, PP[1] - s*g) + (poly_eval*e(g,g)))

True

### 2.5.5 What if $\alpha$ is leaked?
In that case, we could forge a polynomial that passes the proof without knowing the message at all...
In fact, we have not deleted `alpha`, so we can actually act as bad actors and forge said polynomial!

In [22]:
print(f"Oh no, `alpha` leaked: {alpha}")

Oh no, `alpha` leaked: 12816


So we know the original commitment:

In [23]:
print(f"Original commitment: {commitment}")

Original commitment: 29729


Since the commitment is $g^{\phi(\alpha)}$, and $g$ is publicly known, we can also know a value for $\phi(\alpha)$

In [24]:
commitment/g

40538

Now we just need a polynomial that passes through $(\alpha, \phi(\alpha))$... Just to make it nice, let's compute it as a quadratic function

In [25]:
def make_faux_poly(F_p_X, alpha, phi_alpha, s, phi_s):
    xs = [alpha, s]
    ys = [phi_alpha, phi_s]

    F_p = F_p_X.base_ring()

    while (len(xs) != 3):
        x = F_p.random_element()
        if not (x in xs):
            xs.append(x)
            ys.append(F_p.random_element())
    return F_p_X.lagrange_polynomial(zip(xs,ys))

In [26]:
faux_poly = make_faux_poly(F_p_X, alpha, commitment/g, s, poly_eval)
faux_poly

41411*x^2 + 11139*x + 44042

Let's run the protocol with this faux polynomial that generates an identical commitment

In [27]:
faux_poly_eval = faux_poly(s)
print(f"The evaluation of s on the faux polynomial is {faux_poly_eval}")

faux_proof_poly = ((faux_poly - faux_poly_eval)/(x - s)).numerator()
print(f"The faux proof polynomial {faux_proof_poly}")

faux_opening_commit = exponent_evaluation(PP, proof_poly, g)
opening_triplet = (s, faux_poly_eval, faux_opening_commit)
print(f"The faux opening triplet is {opening_triplet}")

faux_verification = e(commitment, g) == (e(faux_opening_commit, PP[1] - s*g) + (faux_poly_eval*e(g,g)))
print(
f"""
Did we deceive the system and verified the faux polynomial {faux_poly}
as if it was the true polynomial {msg_poly}?\n\n{faux_verification}!!
"""
)

The evaluation of s on the faux polynomial is 50946
The faux proof polynomial 41411*x + 30627
The faux opening triplet is (32493, 50946, 11475)

Did we deceive the system and verified the faux polynomial 41411*x^2 + 11139*x + 44042
as if it was the true polynomial 38784*x^20 + 29502*x^19 + 27814*x^18 + 15919*x^17 + 4780*x^16 + 35151*x^15 + 16478*x^14 + 40979*x^13 + 24598*x^12 + 21339*x^11 + 50472*x^10 + 6735*x^9 + 8685*x^8 + 9664*x^7 + 13815*x^6 + 14359*x^5 + 8374*x^4 + 37855*x^3 + 11600*x^2 + 18403*x + 26889?

True!!



# 3. Closing remarks

## 3.1 Recovering the message from the polynomial?

Did you wonder how can we recover the original message from the polynomial?

In [28]:
def decode_poly(poly):
    images = []
    for i in range(1, poly.degree() + 2):
        images.append(poly(i))

    rs = RSCodec(10)
    
    return bytes(rs.decode(images)[0]).decode("utf-8")

In [29]:
decode_poly(msg_poly)

'Hello Zepp!'

Ever wondered what our faux polynomial was encoding?

In [30]:
try:
    print(decode_poly(faux_poly))
except:
    print("Exception!\nEncoded data makes no sense as a reed-solomon encoding")

Exception!
Encoded data makes no sense as a reed-solomon encoding


## 3.2 Abstraction!

Let's abstract all of this into a class

In [31]:
from KZG import KZG

Cool! Let's do all of the previous work again

In [32]:
# Initialize the helper class
kzg = KZG(leak=True)
kzg

KZG commitment class with Finite Field of size 44939 whose generator is 2
Toxic waste has NOT been deleted. Proceed with caution

Let's create the polynomial encoding our message `"Hello Zepp"` and commit it to the KZG proving scheme

In [33]:
msg = message_as_coords("Hello Zepp!")
msg_poly_again = kzg.polynomial_ring()[0].lagrange_polynomial(msg)
commitment_again = kzg.make_commitment(msg_poly_again)
commitment_again

7170

Let's create now the opening triplet. We'll choose an arbitrary `s`

In [34]:
s_again = kzg.F_p.random_element()
s_again

26764

In [35]:
opening_triplet_again = kzg.make_opening_triplet(s_again, msg_poly_again)
opening_triplet_again

(26764, 28067, 36761)

Let's verify it!

In [36]:
kzg.verify(commitment_again, opening_triplet_again)

True

Let's forge a proof again

In [37]:
faux_poly_again = make_faux_poly(
    kzg.polynomial_ring()[0],
    kzg.alpha,
    commitment_again/kzg.g,
    *opening_triplet_again[:2]
)
faux_poly_again

18795*x^2 + 30195*x + 12801

In [38]:
faux_commitment_again = kzg.make_commitment(faux_poly_again)
faux_commitment_again

7170

In [39]:
faux_opening_triplet = kzg.make_opening_triplet(s_again, faux_poly_again)
faux_opening_triplet

(26764, 28067, 36761)

In [40]:
kzg.verify(faux_commitment_again, faux_opening_triplet)

True